In [15]:
import os
import sys
from pathlib import Path
os.chdir('/Users/mac/AIworkspace/LLMWORKSPACE/RAG_Rec')
# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
from dotenv import load_dotenv
load_dotenv()
import torch

In [18]:
os.environ['HF_TOKEN']=os.getenv('HF_TOKEN')

In [16]:
# 켜기
device = torch.device("mps")

In [ ]:
# 끄기
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
print(torch.cuda.is_available())  # False 출력


In [3]:
import os
print(os.getcwd())

/Users/mac/AIworkspace/LLMWORKSPACE/RAG_Rec


In [4]:
import pandas as pd
import ast

# file_path = ('data/movie_record_per_user.csv')
file_path='data/train_movie.csv'
data = pd.read_csv(file_path)
# 'movie_explain' 열을 리스트로 변환
data['movie_explain'] = data['movie_explain'].apply(ast.literal_eval)

In [5]:
data

,UserId,movie_explain
0,1,"[3186 (Drama) ratings: 4, 1270 (Comedy|Sci-Fi)..."
1,2,"[1198 (Action|Adventure) ratings: 4, 1210 (Act..."
2,3,"[593 (Drama|Thriller) ratings: 3, 2858 (Comedy..."
3,4,[1210 (Action|Adventure|Romance|Sci-Fi|War) ra...
4,5,"[2717 (Comedy|Horror) ratings: 1, 908 (Drama|T..."
...,...,...
6035,6036,"[1721 (Drama|Romance) ratings: 4, 2428 (Horror..."
6036,6037,"[1882 (Action|Sci-Fi) ratings: 1, 3508 (Wester..."
6037,6038,"[920 (Drama|Romance|War) ratings: 3, 3396 (Chi..."
6038,6039,"[282 (Drama) ratings: 4, 111 (Drama|Thriller) ..."


# 1. 데이터 벡터db에 저장

In [6]:

# 2. 청킹 함수
def chunk_interactions_with_minimum(interactions, rating_threshold=2, min_items=10):
    """
    평점 변화 및 장르 변화 기준으로 청킹 후, 최소 10개의 상호작용이 포함되도록 조정.

    Args:
        interactions (list): 사용자 상호작용 리스트.
        rating_threshold (int): 평점 변화 임계값.
        min_items (int): 최소 상호작용 개수.

    Returns:
        list: 최소 상호작용 개수를 만족하는 청킹된 리스트.
    """
    chunks = []  # 청킹 결과 저장
    current_chunk = [interactions[0]]  # 첫 번째 상호작용으로 초기화

    for i in range(1, len(interactions)):
        prev = interactions[i - 1]
        current = interactions[i]

        # 이전 및 현재 상호작용의 평점과 장르 추출
        prev_rating = int(prev.split('ratings: ')[1])
        current_rating = int(current.split('ratings: ')[1])
        prev_genres = set(prev.split(' (')[1].split(')')[0].split('|'))
        current_genres = set(current.split(' (')[1].split(')')[0].split('|'))

        # 조건 1: 장르가 포함되지 않은 경우
        genre_disjoint = len(prev_genres & current_genres) == 0

        # 조건 2: 평점 변화가 임계값 이상인 경우
        rating_diff = abs(current_rating - prev_rating) >= rating_threshold

        # 조건을 만족하면 청킹
        if genre_disjoint or rating_diff:
            if len(current_chunk) >= min_items:
                chunks.append(current_chunk)  # 현재 청크 저장
                current_chunk = []  # 새로운 청크 시작

        current_chunk.append(current)

    # 마지막 청크 추가
    if current_chunk:
        chunks.append(current_chunk)

    # 최소 상호작용 개수 보장
    if len(chunks) > 1:
        merged_chunks = []
        temp_chunk = []

        for chunk in chunks:
            temp_chunk.extend(chunk)
            if len(temp_chunk) >= min_items:
                merged_chunks.append(temp_chunk)
                temp_chunk = []

        if temp_chunk:  # 남아 있는 항목 처리
            if merged_chunks:
                merged_chunks[-1].extend(temp_chunk)
            else:
                merged_chunks.append(temp_chunk)

        return merged_chunks

    return chunks  # 청킹 결과 반환

In [7]:

all_chunks = []  # 모든 청크를 저장할 리스트
metadata_list = []  # 메타데이터 저장 리스트
chunked_by_user = {}  # 사용자별 청킹 결과 저장

# 사용자별로 청킹 수행
for _, row in data.iterrows():
    user_id = row['UserId']  # 사용자 ID
    interactions = row['movie_explain']  # 상호작용 리스트

    # 청킹 수행
    chunks = chunk_interactions_with_minimum(interactions, rating_threshold=2, min_items=5)
    # chunks=chunk_interactions(interactions, rating_threshold=3)
    # 사용자별로 청크 저장
    chunked_by_user[user_id] = chunks

    # 각 청크에 대해 메타데이터 추가
    for index, chunk in enumerate(chunks):
        chunk_text = " ".join(chunk)  # 리스트를 문자열로 변환
        all_chunks.append(chunk_text)
        metadata_list.append({"UserId": user_id, "chunk_index": index})  # 사용자 ID와 청크 번호 추가


- 임베딩 및 vectordb 저장

In [13]:
from transformers import AutoTokenizer, AutoModel
import torch
import chromadb

class CustomEmbedding:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.to(self.device)
    
    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    def get_embeddings(self, texts):
        if isinstance(texts, str):
            texts = [texts]
            
        encoded_input = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        ).to(self.device)
        
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        
        embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
        
        return embeddings.cpu().numpy()

def save_to_vectorstore_with_batches(all_chunks, metadata_list, collection_name="movie_interactions", batch_size=100):
    client = chromadb.Client()
    
    try:
        client.delete_collection(collection_name)
    except:
        pass
    collection = client.create_collection(name=collection_name)
    
    embedding_model = CustomEmbedding()

    for i in range(0, len(all_chunks), batch_size):
        batch_texts = all_chunks[i:i + batch_size]
        batch_metadata = metadata_list[i:i + batch_size]

        embeddings = embedding_model.get_embeddings(batch_texts)
        ids = [f"chunk_{j}" for j in range(i, i + len(batch_texts))]

        collection.add(
            embeddings=embeddings.tolist(),
            documents=batch_texts,
            metadatas=batch_metadata,
            ids=ids
        )
    
    print(f"벡터 저장소 생성 완료: {collection.count()} 개의 청크가 저장됨")
    return collection


# 사용 예시
if __name__ == "__main__":
    # 이미 준비된 all_chunks와 metadata_list를 사용하여 벡터 저장소 생성
    collection = save_to_vectorstore_with_batches(all_chunks, metadata_list)


/opt/anaconda3/envs/kmy_env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


KeyboardInterrupt: 

## 1. header 생성 

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from typing import List, Dict
import os

class Llama2Generator:
    def __init__(self, model_name="meta-llama/Llama-3.1-8B"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto",
            token = os.getenv('HF_TOKEN')
        )
        
    def generate_response(self, prompt: str, max_length: int = 512, temperature: float = 0.7) -> str:
        # Llama2 채팅 형식에 맞게 프롬프트 포맷팅
        chat_format = f"[INST] {prompt} [/INST]"
        
        # 입력 인코딩
        inputs = self.tokenizer(chat_format, return_tensors="pt", truncation=True)
        inputs = inputs.to(self.model.device)
        
        # 생성 파라미터 설정
        generation_config = {
            "max_length": max_length,
            "temperature": temperature,
            "do_sample": True,
            "top_p": 0.95,
            "top_k": 50,
            "num_return_sequences": 1,
            "pad_token_id": self.tokenizer.eos_token_id
        }
        
        # 텍스트 생성
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                **generation_config
            )
        
        # 생성된 텍스트 디코딩 및 프롬프트 제거
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = generated_text.split("[/INST]")[-1].strip()
        
        return response

def summarize_user_preferences(user_id: int, interactions: list, llm_model: Llama2Generator) -> str:
    prompt = f"""
INSTRUCTIONS:
Analyze the following user's movie interaction records, which are provided in chronological order. Each record includes the movie's genre and the user's rating.
Your task is to provide a detailed but concise natural language summary of the following:
1. The user's overall preferences in terms of movie genres and their most frequently rated genres.
2. Any notable patterns or shifts in the user's preferences over time (e.g., transitioning from one genre to another, increasing or decreasing ratings for specific genres).
3. The user's general rating behavior (e.g., higher ratings for certain genres, average rating trends, or periods of significant rating changes).
Ensure the summary reflects the temporal nature of the data, highlighting trends or key transitions in the user's preferences and behavior.

USER MOVIE RECORDS:
{interactions}

OUTPUT FORMAT:
<Summary>
"""
    return llm_model.generate_response(prompt, max_length=300, temperature=0.7)

def process_chunks_with_summaries(data, llm_model: Llama2Generator):
    all_chunks_with_headers = []
    metadata_list_with_headers = []
    
    for _, row in data.iterrows():
        user_id = row['UserId']
        interactions = row['movie_explain']
        
        # 청킹 수행 (이미 구현된 함수 사용)
        chunks = chunk_interactions_with_minimum(interactions, rating_threshold=2, min_items=10)
        
        # 각 청크에 대해 처리
        for index, chunk in enumerate(chunks):
            try:
                # LLM으로 취향 및 변화 요약
                summary = summarize_user_preferences(user_id, chunk, llm_model)
                
                # 청크에 헤더 추가
                header = f"[UserId: {user_id}] {summary}"
                chunk_text = f"{header}\n{' '.join(chunk)}"
                
                all_chunks_with_headers.append(chunk_text)
                metadata_list_with_headers.append({"UserId": user_id, "chunk_index": index})
                
            except Exception as e:
                print(f"Error processing chunk for user {user_id}, chunk {index}: {str(e)}")
                continue
    
    return all_chunks_with_headers, metadata_list_with_headers

# 사용 예시
if __name__ == "__main__":
    # Llama2 모델 초기화
    llm_model = Llama2Generator()
    
    # 데이터 처리
    all_chunks_with_headers, metadata_list_with_headers = process_chunks_with_summaries(
        data, 
        llm_model
    )
    
    print(f"처리된 총 청크 수: {len(all_chunks_with_headers)}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

KeyboardInterrupt: 

# 3. vectorstore 불러오기 

In [ ]:
# 정답
file_path_test='data/test_movie.csv'
purchase_history=data.iloc[5]['movie_explain']
df_test=pd.read_csv(file_path_test)
df_test.iloc[5].movie_explain


In [ ]:
import chromadb

class SimilaritySearch:
    def __init__(self, collection_name="movie_interactions"):
        self.client = chromadb.Client()
        self.collection = self.client.get_collection(collection_name)
        self.embedding_model = CustomEmbedding()
    
    def search(self, query: str, k: int = 10) -> dict:
        """
        주어진 쿼리와 가장 유사한 문서를 검색합니다.
        
        Args:
            query (str): 검색할 쿼리 문자열
            k (int): 반환할 결과의 개수
            
        Returns:
            dict: 검색 결과 (문서, 메타데이터, 거리 포함)
        """
        # 쿼리 임베딩 생성
        query_embedding = self.embedding_model.get_embeddings(query)
        
        # 유사도 검색 수행
        results = self.collection.query(
            query_embeddings=query_embedding.tolist(),
            n_results=k
        )
        
        return results

def format_search_results(results: dict) -> str:
    """검색 결과를 보기 좋게 포맷팅합니다."""
    formatted_output = []
    
    for i in range(len(results['ids'][0])):
        formatted_output.append(f"\n--- Result {i+1} ---")
        formatted_output.append(f"Document: {results['documents'][0][i][:200]}...")  # 처음 200자만 표시
        formatted_output.append(f"Distance: {results['distances'][0][i]}")
        formatted_output.append(f"Metadata: {results['metadatas'][0][i]}")
    
    return "\n".join(formatted_output)

# 사용 예시
if __name__ == "__main__":
    # 검색 엔진 초기화
    searcher = SimilaritySearch()
    
    # 구매 이력에서 마지막 항목으로 쿼리 생성
    query = " ".join(purchase_history[-1:])  # purchase_history는 이미 정의되어 있다고 가정
    
    # 검색 수행
    results = searcher.search(query, k=10)
    
    # 결과 출력
    print(f"Query: {query}")
    print(format_search_results(results))
    
    # 전체 결과를 하나의 문자열로 결합
    full_content = "\n".join([doc for doc in results['documents'][0]])
    print("\nFull Content of Retrieved Documents:")
    print(full_content)


# 4. contextual enrichment

# 실험